In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests


In [2]:
# 선수 ID 지정 (예: 11310은 특정 선수의 고유 번호/등판기록이 없는 선수 예 id: 16251)
player_id = "11310" #데이터 프레임에서 가져온 변수가 될 예정
url = f"https://statiz.sporki.com/player/?m=year&p_no={player_id}"
url_nodata ='https://statiz.sporki.com/stats/?m=team&m2=pitching'


In [3]:
def crawl_statiz_pitcher_data(url):
    """
    스탯티즈에서 선발투수 등판 기록을 크롤링하는 함수
    """
    tempValue = ''     #rowspan이 2로 행이 묶여 정보가 누락되는 문제 해결을 위한 변수들
    onRowapply1 = False #한 행의 열데이터 들을 수집하면서 rowspan==2를 감지하면 True
    onRowapply2 = False #한 행의 열데이터를 모두 수집하였을때 onRowapply1이 True면 True
    j=0
    try:
        
        # HTTP 요청 헤더 설정 (봇 차단 방지)
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.7151.104 Safari/537.36'
        }
        
        # 웹페이지 요청
        response = requests.get(url, headers=headers)
        response.raise_for_status()
        
        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # 테이블 찾기 (일반적으로 통계 데이터는 table 태그 안에 있음)
        table = soup.find('table')
        if not table:
            print("테이블을 찾을 수 없습니다.")
            return None
        
        # 헤더 추출 (th 태그 또는 첫 번째 tr의 td 태그)
        headers = []
        header_row = table.find('tr')
        if header_row:
            header_cells = header_row.find_all(['th', 'td'])
            headers = [cell.get_text(strip=True) for cell in header_cells]
        
        # 데이터 행 추출
        data_rows = []
        rows = table.find_all('tr')[1:]  # 첫 번째 행(헤더) 제외
        for i in range (len(list(rows))-1): #마지막 통산 기록 부분 제외를 위해 -1
            row = rows[i]
            cells = row.find_all('td')
            if cells:  # td 태그가 있는 행만 처리
                row_data = []
                
                while j <= (len(list(cells))-1):
                    cell = cells[j]
                    if onRowapply2 ==True:
                        text= tempValue
                        onRowapply1 = False
                        onRowapply2 = False
                        j-=1 #rowspan==2로 인해 누락된 데이터를 입력하느라 원래 데이터를 누락하지 않도록 함
                    else:
                        # 셀 내용 추출 (줄바꿈 제거, 공백 정리)
                        text = cell.get_text(separator=' ', strip=True)

                    # 추가 줄바꿈 문자 제거
                    text = text.replace('\n', ' ').replace('\r', '').strip()
                    # 연속된 공백을 하나로 통일
                    text = ' '.join(text.split())
                    row_data.append(text)
                    # 첫번째 컬럼의 td row span 속성값이 2이상일 경우 값을 저장하고 onRowapply1을 True로 변경
                    if i==1 and j==0:
                        tmpRow = cell.attrs["rowspan"]
                        if int( cell["rowspan"] ) >= 2:
                            tempValue = text
                            onRowapply1 = True #rowspan==2를 감지하면 True로 변경
                    j+=1
                j=0
                
                if row_data:  # 빈 행이 아닌 경우만 추가
                    data_rows.append(row_data)
            if onRowapply1== True: #누락된 데이터를 바로 다음 열에 입력하지 않도록 onRowapply2를 이용
                onRowapply2= True
        
        return headers, data_rows
        
    except requests.RequestException as e:
        print(f"웹페이지 요청 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"데이터 추출 중 오류 발생: {e}")
        return None

In [4]:
def crawl_statiz_pitcher_nodata(url_nodata):
    """
    결측치 처리를 위한 함수
    스탯티즈에서 선발투수 등판 기록이 없을때 리그 평균 스탯을 크롤링하는 함수
    """
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/137.0.7151.104 Safari/537.36'
        }

        # 웹페이지 요청
        response = requests.get(url_nodata, headers=headers)
        response.raise_for_status()

        # BeautifulSoup 객체 생성
        soup = BeautifulSoup(response.text, 'html.parser')

        # 테이블 찾기 (일반적으로 통계 데이터는 table 태그 안에 있음)
        table = soup.find('table')
        if not table:
            print("테이블을 찾을 수 없습니다.")
            return None
        #colum데이터 추출
        headers1 = []
        header_row1 = table.find('tr')
        if header_row1:
            header_cells1 = header_row1.find_all([ 'th','td'])
            headers1 = [cell.get_text(strip=True) for cell in header_cells1]
        #league_colum_DF = pd.DataFrame(headers1)
        # 리그 시즌기록 데이터 추출 (tbody태그 내부에 데이터 존재)
        headers2 = []
        header_row2 = table.find('tbody')
        if header_row2:
            header_cells2 = header_row2.find_all([ 'th','td'])
            headers2 = [cell.get_text(strip=True) for cell in header_cells2]
        #nostat_pitcher_DF = nostat_pitcher_DF.transpose()


        return headers1,headers2

    except requests.RequestException as e:
        print(f"웹페이지 요청 중 오류 발생: {e}")
        return None
    except Exception as e:
        print(f"데이터 추출 중 오류 발생: {e}")
        return None

###원본코드 보호

In [ ]:
columlist = ['ERA', 'FIP', 'WHIP']
columlist1 = ['ERA1', 'FIP1', 'WHIP1']
columlist2 = ['ERA2', 'FIP2', 'WHIP2']

picher_stat10_list = []

for i in range(5):
    picher_stat2_list=[]
    #원정 투수 스탯
    
    player_id=test_data_df[i][4]
    picher_data=crawl_statiz_pitcher_data(url)

    if picher_data[1]==[]:
        picher_nodata=crawl_statiz_pitcher_nodata(url_nodata)
        picher_nodata_DF=pd.DataFrame(picher_nodata[1])
        picher_nodata_DF = picher_nodata_DF.set_axis(picher_nodata[0], axis=0).transpose()
        picher_stat_DF = picher_nodata_DF

    else:
        picherdata_DF=pd.DataFrame(picher_data[1])
        picher_data_DF = picherdata_DF.set_axis(picher_data[0], axis=1)
        picher_stat_DF = picher_data_DF
    
    picher_stat_DF.iloc[-1]
    picher_stat_DF = pd.DataFrame(picher_stat_DF.iloc[-1])

    picher_stat_DF = picher_stat_DF[columlist]
    picher_stat_DF.columns = columlist1

    picher_stat2_list.append(picher_stat_DF)

    player_id=test_data_df[i][5]
    picher_data=crawl_statiz_pitcher_data(url)

    if picher_data[1]==[]:
        picher_nodata=crawl_statiz_pitcher_nodata(url_nodata)
        picher_nodata_DF=pd.DataFrame(picher_nodata[1])
        picher_nodata_DF = picher_nodata_DF.set_axis(picher_nodata[0], axis=0).transpose()
        picher_stat_DF = picher_nodata_DF

    else:
        picherdata_DF=pd.DataFrame(picher_data[1])
        picher_data_DF = picherdata_DF.set_axis(picher_data[0], axis=1)
        picher_stat_DF = picher_data_DF
    
    picher_stat_DF.iloc[-1]
    picher_stat_DF = pd.DataFrame(picher_stat_DF.iloc[-1])

    picher_stat_DF = picher_stat_DF[columlist]
    picher_stat_DF.columns = columlist2

    picher_stat2_list.append(picher_stat_DF)

    picher_stat10_list.append(picher_stat2_list)

picher_stat10_DF=pd.DataFrame(picher_stat10_list)



NameError: name '데이터프레임' is not defined

In [ ]:
# picher_data=crawl_statiz_pitcher_data(url)

# if picher_data[1]==[]:
#     picher_nodata=crawl_statiz_pitcher_nodata(url_nodata)
#     picher_nodata_DF=pd.DataFrame(picher_nodata[1])
#     picher_nodata_DF = picher_nodata_DF.set_axis(picher_nodata[0], axis=0).transpose()
#     picher_stat_DF = picher_nodata_DF

# else:
#     picherdata_DF=pd.DataFrame(picher_data[1])
#     picher_data_DF = picherdata_DF.set_axis(picher_data[0], axis=1)
#     picher_stat_DF = picher_data_DF
#     # picherdata_DF.iloc[-1]



In [ ]:
# picher_stat_DF.iloc[-1]
# picher_stat_DF = pd.DataFrame(picher_stat_DF.iloc[-1])
# picher_stat_DF=picher_stat_DF.transpose()



In [ ]:
# picher_stat_DF=picher_stat_DF.transpose()

In [ ]:
# columlist = ['ERA', 'FIP', 'WHIP']
# columlist1 = ['ERA1', 'FIP1', 'WHIP1']
# columlist2 = ['ERA2', 'FIP2', 'WHIP2']
# picher_stat_DF = picher_stat_DF[columlist]

In [ ]:
# picher_stat_DF.columns = columlist1

In [ ]:
# picher_stat_DF

,ERA1,FIP1,WHIP1
11,4.34,3.57,1.40


In [ ]:
# picher_stat2_list=[]
# # picher_stat2_list.append(picher_stat_DF.columns =[])
# picher_stat2_list.append(picher_stat_DF)

In [ ]:
# picher_stat10_list=[]
# picher_stat10_list.append(picher_stat2_list)
# picher_stat10_list

[[     ERA   FIP  WHIP
  11  4.34  3.57  1.40,
       ERA   FIP  WHIP
  11  4.34  3.57  1.40]]